In [66]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit, GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from datetime import datetime
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import graphviz
from sklearn.tree import export_graphviz

In [67]:
df = pd.read_csv('combinedData.csv')

Encoding

In [68]:
id_features = ['tmID','playerID', 'college', 'collegeOther', 'coachID']
categorical_features = [ 'pos', 'award', 'confID', 'results']
df = pd.get_dummies(df, columns=categorical_features)


In [69]:
for feature in id_features:
    label_encoder = LabelEncoder()
    df[f'{feature}_encoded'] = label_encoder.fit_transform(df[feature])

df['playoff'] = df['playoff'].replace({'N': 0, 'Y': 1})

df = df.drop(columns=id_features)

In [70]:
#YYYY-MM-DD to unix
df['birthDataUnix'] = pd.to_datetime(df['birthDate']).apply(lambda x: int(x.timestamp()))
df = df.drop(columns=['birthDate'])

df_original=df
feature_cols = [col for col in df.columns if col not in ['playoff']]

# Data Leakage

features not to replace with last years:
 - tmID
 - year
 - playerID
 - stint
 - pos
 - height
 - weight
 - college
 - college other
 - birthDate
 - age
 - coachId
 - coachStint
 - confID

In [71]:
features_to_replace = [col for col in feature_cols if col not in [
    'tmID', 'year', 'playerID', 'stint', 'pos', 'height', 'weight', 'college',
    'collegeOther', 'birthDate', 'age', 'coachID', 'coachStint', 'confID',
    'tmID_encoded', 'playerID_encoded', 'college_encoded','collegeOther_encoded',
    'coachID_encoded','birthDataUnix','confID_EA','confID_WE','pos_C','pos_C-F',
    'pos_F','pos_F-C','pos_F-G','pos_G','pos_G-F']]

print(features_to_replace)

['GP', 'GS', 'minutes', 'points', 'oRebounds', 'dRebounds', 'rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'PF', 'fgAttempted', 'fgMade', 'ftAttempted', 'ftMade', 'threeAttempted', 'threeMade', 'dq', 'mpg', 'ppg', 'apg', 'rpg', 'spg', 'bpg', 'topg', 'pfpg', 'fg%', '3p%', 'ft%', 'orpg', 'drpg', 'pointsFromFieldGoal', 'percentage_pointsFromFieldGoal', 'percentage_pointsFromFreeThrow', 'percentage_pointsFromThree', 'coachWon', 'coachLost', 'coach W/L Ratio', 'seed', 'o_fga', 'o_fta', 'o_3pa', 'o_oreb', 'o_dreb', 'o_reb', 'o_asts', 'o_pf', 'o_stl', 'o_to', 'o_blk', 'o_pts', 'd_fgm', 'd_fga', 'd_ftm', 'd_fta', 'd_3pm', 'd_3pa', 'd_oreb', 'd_dreb', 'd_reb', 'd_asts', 'd_pf', 'd_stl', 'd_to', 'd_blk', 'd_pts', 'win_ratio', 'homeW_ratio', 'awayW_ratio', 'min', 'o_fg%', 'o_ft%', 'o_3p%', 'd_fg%', 'd_ft%', 'd_3p%', 'coachAward', 'award_ASGMVP', 'award_DPOTY', 'award_KPSA', 'award_MIP', 'award_MVP', 'award_ROTY', 'award_SWOTY', 'award_WADT', 'award_WADTHM', 'award_WFMVP', 'results_Unknow

In [72]:
label_encoder = LabelEncoder()
accuracies = []
model = DecisionTreeClassifier()

for i in range(2, 11):
    train_years = range(1,i)
    test_year = i

    train_data = df[df['year'].isin(train_years)]
    #print("============================================================================")
    #print(train_data)
    replace_data = df[df['year'] == test_year-1]
    test_data = df[df['year'] == test_year]
    #print(test_data)
    #print("============================================================================")

    # replace data with last years data
    key_columns = ['tmID_encoded', 'playerID_encoded']
    
    for index, current_row in test_data.iterrows():
        common_key = tuple(current_row[key_columns])

        # Check if there's a matching entry in the previous year's dataframe
        matching_entry = replace_data[(replace_data[key_columns[0]] == current_row[key_columns[0]]) & (replace_data[key_columns[1]] == current_row[key_columns[1]])]

        if not matching_entry.empty:
            # Replace values in columns_to_replace with data from the matching entry
            for column in features_to_replace:
                test_data.at[index, column] = matching_entry.iloc[0][column]
        else:
            # Remove the row if there is no matching entry
            test_data.drop(index, inplace=True)
    
    print(test_data)


    y_train = train_data['playoff']
    y_test = test_data['playoff']

    # Prepare the data for training and testing using unaggregated feature data (X)
    X_train, X_test = train_data[feature_cols], test_data[feature_cols]

    # Train your model (e.g., Random Forest)
    model.fit(X_train, y_train)

    # Make predictions for individual players
    player_predictions = model.predict(X_test)


    # Evaluate the model using the encoded labels
    accuracy = accuracy_score(y_test, player_predictions)
    accuracies.append(accuracy)


/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(index, inplace=True)
/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(index, inplace=True)
/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(index, inplace=True)
/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is t

/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(index, inplace=True)
/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(index, inplace=True)
/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(index, inplace=True)
/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is t

      year  stint  GP  GS  minutes  points  oRebounds  dRebounds  rebounds  \
43       2      0  12   9      323     139         21         24        45   
44       2      0  29   0      275      92         26         37        63   
48       2      0  30   7      659     156         29         77       106   
49       2      0  32  32     1099     282         21         56        77   
50       2      0  32  32     1123     565         35        101       136   
...    ...    ...  ..  ..      ...     ...        ...        ...       ...   
1999     2      0  32  32     1131     561         57        183       240   
2001     2      0  32  32     1046     497         22         34        56   
2003     2      0  31   0      295     130         39         41        80   
2004     2      0  32  32     1046     314         79        129       208   
2007     2      0  19   0      103      26          6          8        14   

      assists  ...  results_label1  results_label2  results_lab

/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(index, inplace=True)
/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(index, inplace=True)
/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(index, inplace=True)
/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is t

      year  stint  GP  GS  minutes  points  oRebounds  dRebounds  rebounds  \
52       3      0  18   0      102      12          3         12        15   
53       3      0  22   0      372     100         11         32        43   
54       3      0  32  11      623     191         48         65       113   
55       3      0  18   0      148      54          8         27        35   
56       3      0  32  32     1007     365         55         98       153   
...    ...    ...  ..  ..      ...     ...        ...        ...       ...   
2017     3      0  20   0      137      34          5          4         9   
2018     3      0  30   0      319      64         40         65       105   
2019     3      0  32  32      989     225         74        103       177   
2020     3      0  25   3      455      76         11         31        42   
2022     3      0  30   2      336     101         19         24        43   

      assists  ...  results_label1  results_label2  results_lab

...

      year  stint  GP  GS  minutes  points  oRebounds  dRebounds  rebounds  \
153      8      0  23  16      454     161          6         21        27   
154      8      0  34  31     1032     467         55        133       188   
157      8      0  27   1      277      58         27         19        46   
158      8      0  30  27      841     282         33         74       107   
162      8      0  15  13      345      50         15         25        40   
...    ...    ...  ..  ..      ...     ...        ...        ...       ...   
2099     8      0  27  27      727     199         18         52        70   
2100     8      0  34  19      885     301         69        134       203   
2101     8      0  34  19      885     301         69        134       203   
2102     8      0  34  34      975     364         13         76        89   
2103     8      0  34  34      975     364         13         76        89   

      assists  ...  results_label1  results_label2  results_lab

/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(index, inplace=True)
/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(index, inplace=True)
/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(index, inplace=True)
/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is t

      year  stint  GP  GS  minutes  points  oRebounds  dRebounds  rebounds  \
163      9      0  30  27      776     259         19         45        64   
165      9      0  33  33     1078     544         79        175       254   
168      9      0  19   0      271      95          6         27        33   
169      9      0  29  25      853     288         66        127       193   
170      9      0  33   5      762     386         30         80       110   
...    ...    ...  ..  ..      ...     ...        ...        ...       ...   
2123     9      0  30   2      455     120         20         29        49   
2124     9      0  28   0      279      83         25         53        78   
2125     9      0  28   0      279      83         25         53        78   
2128     9      0  34  31      994     375        100        142       242   
2129     9      0  34  31      994     375        100        142       242   

      assists  ...  results_label1  results_label2  results_lab

/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(index, inplace=True)
/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(index, inplace=True)
/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(index, inplace=True)
/tmp/ipykernel_14346/1684439921.py:32: SettingWithCopyWarning: 
A value is t

      year  stint  GP  GS  minutes  points  oRebounds  dRebounds  rebounds  \
14      10      0  29  20      671     269         10         55        65   
15      10      0  12   8      277     112         39         40        79   
17      10      0  33  22      605     189         35         54        89   
18      10      0  34  31      960     388         13         59        72   
26      10      1  33  15      745     240         44         95       139   
...    ...    ...  ..  ..      ...     ...        ...        ...       ...   
2140    10      0  34  34      972     403         32        109       141   
2142    10      1   8   7      212      89         10         39        49   
2143    10      0  34   6      529     164         53         83       136   
2146    10      0  24   1      183      48         17         36        53   
2147    10      0  34  28      795     229         77        116       193   

      assists  ...  results_label1  results_label2  results_lab

In [73]:
print(accuracies, ' ')
print(sum(accuracies)/9)

[0.6268656716417911, 0.7044025157232704, 0.27956989247311825, 0.6181818181818182, 0.7454545454545455, 0.8865979381443299, 0.49019607843137253, 0.8617021276595744, 0.46551724137931033]  
0.6309430921210146


In [74]:
dot_data = export_graphviz(model, out_file=None,
                          feature_names=X_train.columns, # Specify your feature names
                          class_names=['No Playoff', 'Playoff'], # Specify your class names
                          filled=True, rounded=True, special_characters=True)

graph = graphviz.Source(dot_data)
graph.render("decision_tree") # This will save the tree as 'decision_tree.pdf'
graph.view("decision_tree")   # This will open the tree in your default PDF viewer


'decision_tree.pdf'